In [71]:
import pandas as pd

df = pd.read_csv('UL-Standards-List.csv')
# drop rows with no description
df = df.dropna(subset=['Description'])
len(df)
# df.head()


2123

In [72]:
# Description to Title Case since there are some descriptions that are all caps
df['Description'] = df['Description'].str.title()





In [73]:

df.iloc[144]['Description']

'Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method) / Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method)'

In [74]:
# Removing duplicates in the description separated by ' / '
def remove_duplicates(string):
    words = string.split(' / ')
    result = []
    for word in words:
        if word not in result:
            result.append(word)
    return ' / '.join(result)  # nothing to join if there are no duplicates

df['Description'] = df['Description'].apply(remove_duplicates)
df.iloc[570]['Description']

In [5]:
df

'Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method)'

In [70]:
df['Category'] = ''
df['Topic'] = ''
def find_words_in_description(row, words):
    start_index = 0
    for word in words:
        i = row['Description'].find(word, start_index)
        if i == -1:
            return False
        start_index = i + len(word)
    return start_index

words = ['Standard', 'Method', 'Test']



for index, row in df.iterrows():
    i = row['Description'].find('Standard')

    if i != -1:
        df.at[index, 'Category'] = 'Standard'
        split_description = row['Description'][i+8:].split(' For ', maxsplit=1)
        if len(split_description) > 1:
            print(split_description)
            df.at[index, 'Topic'] = split_description[1]
    else:
        df.at[index, 'Description'] = row['Description']
df
#     if i != -1:
#         df.at[index, 'Category'] = 'Standard'
#         df.at[index, 'Description'] =  row['Description'][i+9:].split(' for ')[1]
#     else:
#         df.at[index, 'Description'] = row['Description']
# df

Standard For Flexible Metal Conduit   8
Standard For Armored Cable   8
Standard For Surface Metal Raceways And Fittings   8
Nonmetallic Surface Raceways And Fittings   None
Standard For Strut-Type Channel Raceways And Fittings   8
Standard For Surface Raceways And Fittings For Use With Data, Signal, And Control Circuits   8
Electrical Rigid Metal Conduit - Steel   None
Standard For Electrical Rigid Metal Conduit - Aluminum And Stainless Steel   8
Outline Of Investigation For Pump Tank Extinguishers   7
Water Based Agent Fire Extinguishers   None
Standard For Fire Tests Of Window Assemblies   8
Subject 10 - Fire Doors   None
Guide For Evaluating Doors And Frames For Listing And Labelling   5
Standard For Tin-Clad Fire Doors   8
Standard For Fire Tests Of Door Assemblies   8
Seguridad Contra Incendio – Conjuntos De Montaje De Puertas Contra Incendio – Método De Prueba De Presión Positiva.   None
Standard For Positive Pressure Fire Tests Of Door Assemblies   8
Standard For Fire Tests Of F

In [4]:
# from top2vec import Top2Vec
# model = Top2Vec(documents=df['Description'].tolist())



KeyboardInterrupt

